In [ ]:
!pip install opencv-python-headless networkx matplotlib
from google.colab import files
uploaded = files.upload()
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


image_name = next(iter(uploaded))
img = cv2.imread(image_name)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title("Planta padronizada")
plt.axis('off')
plt.show()

# Configurações de cores
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

azul_baixo = np.array([100, 150, 50])
azul_alto = np.array([140, 255, 255])

vermelho_baixo1 = np.array([0, 100, 100])
vermelho_alto1 = np.array([10, 255, 255])
vermelho_baixo2 = np.array([160, 100, 100])
vermelho_alto2 = np.array([180, 255, 255])

verde_baixo = np.array([36, 50, 50])
verde_alto = np.array([86, 255, 255])

amarelo_baixo = np.array([20, 100, 100])
amarelo_alto = np.array([40, 255, 255])

mascara_azul = cv2.inRange(hsv, azul_baixo, azul_alto)
mascara_vermelho = cv2.bitwise_or(
    cv2.inRange(hsv, vermelho_baixo1, vermelho_alto1),
    cv2.inRange(hsv, vermelho_baixo2, vermelho_alto2))
mascara_verde = cv2.inRange(hsv, verde_baixo, verde_alto)
mascara_amarelo = cv2.inRange(hsv, amarelo_baixo, amarelo_alto)

# Parâmetros do grid (10 = 1m)
cell_size = 10
h, w = img.shape[:2]
nodes = []
G = nx.Graph()

# Operações morfológicas para melhorar a detecção
kernel_size = max(3, cell_size // 3)
kernel = np.ones((kernel_size, kernel_size), np.uint8)

for i, mascara in enumerate([mascara_azul, mascara_vermelho, mascara_verde, mascara_amarelo]):
    # Fechamento para preencher pequenas lacunas
    cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel, mascara, iterations=2)
    # Abertura para remover pequenos ruídos
    cv2.morphologyEx(mascara, cv2.MORPH_OPEN, kernel, mascara, iterations=1)

# Cria máscara para áreas pretas (bloqueio de nós)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, mascara_bloqueio = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY_INV)

# Construção do grafo
for y in range(0, h, cell_size):
    for x in range(0, w, cell_size):
        centro_y = min(y + cell_size//2, h-1)
        centro_x = min(x + cell_size//2, w-1)

        if mascara_bloqueio[centro_y, centro_x] == 0:
            node = (x // cell_size, y // cell_size)
            nodes.append(node)
            G.add_node(node)

# Construção dos nós do grafo
for y in range(0, h, cell_size):
    for x in range(0, w, cell_size):
        centro_y = min(y + cell_size//2, h-1)
        centro_x = min(x + cell_size//2, w-1)

        if mascara_bloqueio[centro_y, centro_x] == 0:
            node = (x // cell_size, y // cell_size)
            nodes.append(node)
            G.add_node(node)

# 4. Detecção de Arestas Aprimorada
def verificar_aresta(x1, y1, x2, y2, hsv, cell_size, mascara_bloqueio):
    # Criar máscara para a linha de conexão
    mask = np.zeros((hsv.shape[0], hsv.shape[1]), dtype=np.uint8)
    cv2.line(mask, (x1, y1), (x2, y2), 255, thickness=3)
    
    # Verificar se a linha cruza áreas bloqueadas
    if np.any(cv2.bitwise_and(mascara_bloqueio, mask)):
        return None 
    
    # Analisar cores ao longo da linha
    line_pixels = cv2.bitwise_and(hsv, hsv, mask=mask)
    total_pixels = np.count_nonzero(mask) + 1e-5 
    
    # Verificar cada tipo de elemento
    azul_pixels = cv2.inRange(line_pixels, azul_baixo, azul_alto)
    azul_ratio = np.count_nonzero(azul_pixels) / total_pixels
    
    vermelho_pixels = cv2.bitwise_or(
        cv2.inRange(line_pixels, vermelho_baixo1, vermelho_alto1),
        cv2.inRange(line_pixels, vermelho_baixo2, vermelho_alto2))
    vermelho_ratio = np.count_nonzero(vermelho_pixels) / total_pixels
    
    verde_pixels = cv2.inRange(line_pixels, verde_baixo, verde_alto)
    verde_ratio = np.count_nonzero(verde_pixels) / total_pixels
    
    amarelo_pixels = cv2.inRange(line_pixels, amarelo_baixo, amarelo_alto)
    amarelo_ratio = np.count_nonzero(amarelo_pixels) / total_pixels
    
    # Determinar o peso com base no elemento predominante
    if azul_ratio > 0.15:  # Parede (concreto)
        return 10
    elif vermelho_ratio > 0.15:  # Janela
        return 8
    elif verde_ratio > 0.15:  # Porta
        return 4
    elif amarelo_ratio > 0.15:  # MDF
        return 5
    else:  # Passagem livre
        return 1

# 5. Construção do Grafo Aprimorada
for node in nodes:
    x, y = node
    for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]:
        neighbor = (x + dx, y + dy)
        if neighbor in G:
            # Coordenadas dos centros das células
            x1 = x * cell_size + cell_size // 2
            y1 = y * cell_size + cell_size // 2
            x2 = neighbor[0] * cell_size + cell_size // 2
            y2 = neighbor[1] * cell_size + cell_size // 2
            
            # Verificar se a aresta cruza algum obstáculo
            weight = verificar_aresta(x1, y1, x2, y2, hsv, cell_size, mascara_bloqueio)
            
            if weight is not None:
                G.add_edge(node, neighbor, weight=weight)

# Imprime Grafo
def mostrar_grafo(G, imagem_fundo, cell_size):
    pos = {n: (n[0] * cell_size, n[1] * cell_size) for n in G.nodes()}

    fig = plt.figure(figsize=(12, 10))
    plt.subplots_adjust(right=0.85)
    if imagem_fundo is not None:
        plt.imshow(imagem_fundo)

    weight_colors = {
        1: 'gray',
        4: 'green',
        5: 'yellow',
        8: 'red',
        10: 'blue'
    }

    for (u, v, d) in G.edges(data=True):
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        peso = d['weight']
        cor = weight_colors.get(peso, 'blue')
        plt.plot(x, y, color=cor, linewidth=1 + peso * 0.5, alpha=0.8)

    xs, ys = zip(*pos.values())
    plt.scatter(xs, ys, s=20, color='black', edgecolors='white', zorder=3)

    plt.title("Grafo de navegação")
    plt.axis('off')

    legend_elements = [
        plt.Line2D([0], [0], color='gray', lw=2, label='Passagem livre (1)'),
        plt.Line2D([0], [0], color='green', lw=2, label='Porta (4)'),
        plt.Line2D([0], [0], color='yellow', lw=2, label='MDF (5)'),
        plt.Line2D([0], [0], color='red', lw=2, label='Janela (8)'),
        plt.Line2D([0], [0], color='blue', lw=2, label='Concreto (10)')
    ]

    plt.legend(handles=legend_elements,
               loc='center left',
               bbox_to_anchor=(1.02, 0.5),
               frameon=False)

    plt.show()

#mostrar_grafo(G, None, cell_size)
mostrar_grafo(G, img_rgb, cell_size)